In [66]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np

In [67]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [68]:
df=pd.read_csv('/content/gdrive/MyDrive/fraudTest.csv')

In [69]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

# 2.) Select four columns to use as features (one just be trans_date_trans)

In [83]:
df_select = df[['trans_date_trans_time','category','amt','city_pop',"is_fraud"]].copy()

In [84]:
df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])

In [72]:
df_select.columns

Index(['trans_date_trans_time', 'category', 'amt', 'city', 'is_fraud'], dtype='object')

# 3.) Create a unique variable out of trans_date.

In [85]:
df_select['time_var']=[i.minute for i in df_select['trans_date_trans_time']]

In [86]:
X = pd.get_dummies(df_select, columns=["category"]).drop(['trans_date_trans_time','is_fraud'],axis = 1)
y = df["is_fraud"]

In [87]:
X.head()

,amt,city_pop,time_var,category_entertainment,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,2.86,333497,14,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,29.84,302,14,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,41.28,34496,14,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,60.05,54767,15,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3.19,1126,15,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# 4.) Oversample the data (this will be your training data).

In [76]:
#skip

In [88]:
resample_X = X
resample_y = y

# 5.) Train a Logistic regression.

In [90]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(resample_X)

In [91]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression().fit(X_normalized, resample_y)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [100]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Assume you have already trained your logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Get the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define the desired false negative rate
target_fnr = 0.05

# Get the total number of negative instances
num_negatives = conf_matrix[0][0] + conf_matrix[0][1]

# Get the total number of false negatives
num_false_negatives = conf_matrix[1][0]

# Calculate the threshold at which the false negative rate is less than or equal to the target
threshold = num_false_negatives / num_negatives

# Print the threshold
print(threshold)

0.0038476128542784373


# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [107]:
df_temp = df_select.copy()
df_temp["pred"] = log_reg.predict(resample_X)
df_temp = df_temp[["pred", "is_fraud", "amt"]]


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [108]:
df_temp.head()

,pred,is_fraud,amt
0,0,0,2.86
1,0,0,29.84
2,0,0,41.28
3,0,0,60.05
4,0,0,3.19


In [109]:
df_temp['factor'] = np.where(df_temp['pred'] == df_temp['is_fraud'], 0.02, -1) 
df_temp['ind_profit'] = df_temp['amt']*df_temp['factor']
Profit = sum(df_temp['ind_profit']) 
Profit

-5341377.2929999605

# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [110]:
clf = LogisticRegression(penalty='l1', solver ='saga') 
clf.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(penalty='l1', solver='saga')

In [111]:
from sklearn.metrics import roc_curve
clf.predict_proba(X_test)
y_probs = clf.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1])
threshold_index = np.abs(fpr - 0.05).argmin() 
threshold = thresholds[threshold_index] 
threshold

0.4814234586630003

In [112]:
y_preds = (y_probs[:,1] > threshold).astype(int) 
conf_matrix = confusion_matrix(y_test, y_preds) 
print(conf_matrix)

[[104982   5736]
 [   423      3]]


In [113]:
clf.predict_proba(X_test)

array([[9.18233121e-01, 8.17668791e-02],
       [9.60627976e-01, 3.93720241e-02],
       [5.67935313e-01, 4.32064687e-01],
       ...,
       [9.99999999e-01, 7.43901689e-10],
       [5.63804075e-01, 4.36195925e-01],
       [6.50493596e-01, 3.49506404e-01]])

In [115]:
#data is garbage